In [59]:
# 1.샘플 데이터 소개
import findspark
findspark.find()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Spark") \
    .config("soark.sql.rep1.eagerEval.enabled", True) \
    .getOrCreate()

In [60]:
# 샘플 데이터(지역별 1인세대수)
df = spark.read.option("header", "true").csv("./oneperson_house.csv", encoding = 'euc-kr')
df.show()

+------------+----------+----------+--------+---------------+----+----+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------

In [61]:
# 2.데이터전처리
# 컬럼 제거
df = df.drop("행정기관코드", "기준연월")
df.show()

+----------+--------+---------------+----+----+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-

In [62]:
# DataFrame의 스키마 확인
df.printSchema()

root
 |-- 시도명: string (nullable = true)
 |-- 시군구명: string (nullable = true)
 |-- 읍면동명: string (nullable = true)
 |-- 계: string (nullable = true)
 |-- 남자: string (nullable = true)
 |-- 여자: string (nullable = true)
 |-- 만0세남자: string (nullable = true)
 |-- 만1세남자: string (nullable = true)
 |-- 만2세남자: string (nullable = true)
 |-- 만3세남자: string (nullable = true)
 |-- 만4세남자: string (nullable = true)
 |-- 만5세남자: string (nullable = true)
 |-- 만6세남자: string (nullable = true)
 |-- 만7세남자: string (nullable = true)
 |-- 만8세남자: string (nullable = true)
 |-- 만9세남자: string (nullable = true)
 |-- 만10세남자: string (nullable = true)
 |-- 만11세남자: string (nullable = true)
 |-- 만12세남자: string (nullable = true)
 |-- 만13세남자: string (nullable = true)
 |-- 만14세남자: string (nullable = true)
 |-- 만15세남자: string (nullable = true)
 |-- 만16세남자: string (nullable = true)
 |-- 만17세남자: string (nullable = true)
 |-- 만18세남자: string (nullable = true)
 |-- 만19세남자: string (nullable = true)
 |-- 만20세남자: string (nullable = true)

In [63]:
# 데이터 형변환
# 제외할 컬럼 이름
exclude_columns = ["시도명","시군구명","읍면동명"]

# 특정 컬럼을 제외한 모든 컬럼을 선택하는 변수 생성
cols = [col for col in df.columns if col not in exclude_columns]
for i in cols:
    df = df.withColumn(i, df[i].cast('int'))

In [64]:
# DataFrame의 스키마 확인
df.printSchema()

root
 |-- 시도명: string (nullable = true)
 |-- 시군구명: string (nullable = true)
 |-- 읍면동명: string (nullable = true)
 |-- 계: integer (nullable = true)
 |-- 남자: integer (nullable = true)
 |-- 여자: integer (nullable = true)
 |-- 만0세남자: integer (nullable = true)
 |-- 만1세남자: integer (nullable = true)
 |-- 만2세남자: integer (nullable = true)
 |-- 만3세남자: integer (nullable = true)
 |-- 만4세남자: integer (nullable = true)
 |-- 만5세남자: integer (nullable = true)
 |-- 만6세남자: integer (nullable = true)
 |-- 만7세남자: integer (nullable = true)
 |-- 만8세남자: integer (nullable = true)
 |-- 만9세남자: integer (nullable = true)
 |-- 만10세남자: integer (nullable = true)
 |-- 만11세남자: integer (nullable = true)
 |-- 만12세남자: integer (nullable = true)
 |-- 만13세남자: integer (nullable = true)
 |-- 만14세남자: integer (nullable = true)
 |-- 만15세남자: integer (nullable = true)
 |-- 만16세남자: integer (nullable = true)
 |-- 만17세남자: integer (nullable = true)
 |-- 만18세남자: integer (nullable = true)
 |-- 만19세남자: integer (nullable = true)
 |-- 만20세남자: i

In [65]:
# 컬럼끼리 연산하여 새 컬럼 생성
import pyspark.sql.functions as F
df = df.withColumn('10대 이하', F.col('만0세남자') + F.col('만1세남자') + F.col('만2세남자') + F.col('만3세남자') + F.col('만4세남자') + F.col('만5세남자')
                   + F.col('만6세남자') + F.col('만7세남자') + F.col('만8세남자') + F.col('만9세남자')
                   + F.col('만0세여자') + F.col('만1세여자') + F.col('만2세여자') + F.col('만3세여자') + F.col('만4세여자') + F.col('만5세여자')
                   + F.col('만6세여자') + F.col('만7세여자') + F.col('만8세여자') + F.col('만9세여자'))
df = df.withColumn('10대', F.col('만10세남자') + F.col('만11세남자') + F.col('만12세남자') + F.col('만13세남자') + F.col('만14세남자') + F.col('만15세남자')
                   + F.col('만16세남자') + F.col('만17세남자') + F.col('만18세남자') + F.col('만19세남자')
                   + F.col('만10세여자') + F.col('만11세여자') + F.col('만12세여자') + F.col('만13세여자') + F.col('만14세여자') + F.col('만15세여자')
                   + F.col('만16세여자') + F.col('만17세여자') + F.col('만18세여자') + F.col('만19세여자'))
df = df.withColumn('20대', F.col('만20세남자') + F.col('만21세남자') + F.col('만22세남자') + F.col('만23세남자') + F.col('만24세남자') + F.col('만25세남자')
                   + F.col('만26세남자') + F.col('만27세남자') + F.col('만28세남자') + F.col('만29세남자')
                   + F.col('만20세여자') + F.col('만21세여자') + F.col('만22세여자') + F.col('만23세여자') + F.col('만24세여자') + F.col('만25세여자')
                   + F.col('만26세여자') + F.col('만27세여자') + F.col('만28세여자') + F.col('만29세여자'))
df = df.withColumn('30대', F.col('만30세남자') + F.col('만31세남자') + F.col('만32세남자') + F.col('만33세남자') + F.col('만34세남자') + F.col('만35세남자')
                   + F.col('만36세남자') + F.col('만37세남자') + F.col('만38세남자') + F.col('만39세남자')
                   + F.col('만30세여자') + F.col('만31세여자') + F.col('만32세여자') + F.col('만33세여자') + F.col('만34세여자') + F.col('만35세여자')
                   + F.col('만36세여자') + F.col('만37세여자') + F.col('만38세여자') + F.col('만39세여자'))
df = df.withColumn('40대', F.col('만40세남자') + F.col('만41세남자') + F.col('만42세남자') + F.col('만43세남자') + F.col('만44세남자') + F.col('만45세남자')
                   + F.col('만46세남자') + F.col('만47세남자') + F.col('만48세남자') + F.col('만49세남자')
                   + F.col('만40세여자') + F.col('만41세여자') + F.col('만42세여자') + F.col('만43세여자') + F.col('만44세여자') + F.col('만45세여자')
                   + F.col('만46세여자') + F.col('만47세여자') + F.col('만48세여자') + F.col('만49세여자'))
df = df.withColumn('50대', F.col('만50세남자') + F.col('만51세남자') + F.col('만52세남자') + F.col('만53세남자') + F.col('만54세남자') + F.col('만55세남자')
                   + F.col('만56세남자') + F.col('만57세남자') + F.col('만58세남자') + F.col('만59세남자')
                   + F.col('만50세여자') + F.col('만51세여자') + F.col('만52세여자') + F.col('만53세여자') + F.col('만54세여자') + F.col('만55세여자')
                   + F.col('만56세여자') + F.col('만57세여자') + F.col('만58세여자') + F.col('만59세여자'))
df = df.withColumn('60대', F.col('만60세남자') + F.col('만61세남자') + F.col('만62세남자') + F.col('만63세남자') + F.col('만64세남자') + F.col('만65세남자')
                   + F.col('만66세남자') + F.col('만67세남자') + F.col('만68세남자') + F.col('만69세남자')
                   + F.col('만60세여자') + F.col('만61세여자') + F.col('만62세여자') + F.col('만63세여자') + F.col('만64세여자') + F.col('만65세여자')
                   + F.col('만66세여자') + F.col('만67세여자') + F.col('만68세여자') + F.col('만69세여자'))
df = df.withColumn('70대', F.col('만70세남자') + F.col('만71세남자') + F.col('만72세남자') + F.col('만73세남자') + F.col('만74세남자') + F.col('만75세남자')
                   + F.col('만76세남자') + F.col('만77세남자') + F.col('만78세남자') + F.col('만79세남자')
                   + F.col('만70세여자') + F.col('만71세여자') + F.col('만72세여자') + F.col('만73세여자') + F.col('만74세여자') + F.col('만75세여자')
                   + F.col('만76세여자') + F.col('만77세여자') + F.col('만78세여자') + F.col('만79세여자'))
df = df.withColumn('80대', F.col('만80세남자') + F.col('만81세남자') + F.col('만82세남자') + F.col('만83세남자') + F.col('만84세남자') + F.col('만85세남자')
                   + F.col('만86세남자') + F.col('만87세남자') + F.col('만88세남자') + F.col('만89세남자')
                   + F.col('만80세여자') + F.col('만81세여자') + F.col('만82세여자') + F.col('만83세여자') + F.col('만84세여자') + F.col('만85세여자')
                   + F.col('만86세여자') + F.col('만87세여자') + F.col('만88세여자') + F.col('만89세여자'))
df = df.withColumn('90대', F.col('만90세남자') + F.col('만91세남자') + F.col('만92세남자') + F.col('만93세남자') + F.col('만94세남자') + F.col('만95세남자')
                   + F.col('만96세남자') + F.col('만97세남자') + F.col('만98세남자') + F.col('만99세남자')
                   + F.col('만90세여자') + F.col('만91세여자') + F.col('만92세여자') + F.col('만93세여자') + F.col('만94세여자') + F.col('만95세여자')
                   + F.col('만96세여자') + F.col('만97세여자') + F.col('만98세여자') + F.col('만99세여자'))
df = df.withColumn('100대 이상', F.col('만100세남자') + F.col('만101세남자') + F.col('만102세남자') + F.col('만103세남자') + F.col('만104세남자') + F.col('만105세남자')
                   + F.col('만106세남자') + F.col('만107세남자') + F.col('만108세남자') + F.col('만109세남자') + F.col('만110세이상남자')
                   + F.col('만100세여자') + F.col('만101세여자') + F.col('만102세여자') + F.col('만103세여자') + F.col('만104세여자') + F.col('만105세여자')
                   + F.col('만106세여자') + F.col('만107세여자') + F.col('만108세여자') + F.col('만109세여자') + F.col('만110세이상여자'))

In [66]:
# 컬럼 확인
df.columns

['시도명',
 '시군구명',
 '읍면동명',
 '계',
 '남자',
 '여자',
 '만0세남자',
 '만1세남자',
 '만2세남자',
 '만3세남자',
 '만4세남자',
 '만5세남자',
 '만6세남자',
 '만7세남자',
 '만8세남자',
 '만9세남자',
 '만10세남자',
 '만11세남자',
 '만12세남자',
 '만13세남자',
 '만14세남자',
 '만15세남자',
 '만16세남자',
 '만17세남자',
 '만18세남자',
 '만19세남자',
 '만20세남자',
 '만21세남자',
 '만22세남자',
 '만23세남자',
 '만24세남자',
 '만25세남자',
 '만26세남자',
 '만27세남자',
 '만28세남자',
 '만29세남자',
 '만30세남자',
 '만31세남자',
 '만32세남자',
 '만33세남자',
 '만34세남자',
 '만35세남자',
 '만36세남자',
 '만37세남자',
 '만38세남자',
 '만39세남자',
 '만40세남자',
 '만41세남자',
 '만42세남자',
 '만43세남자',
 '만44세남자',
 '만45세남자',
 '만46세남자',
 '만47세남자',
 '만48세남자',
 '만49세남자',
 '만50세남자',
 '만51세남자',
 '만52세남자',
 '만53세남자',
 '만54세남자',
 '만55세남자',
 '만56세남자',
 '만57세남자',
 '만58세남자',
 '만59세남자',
 '만60세남자',
 '만61세남자',
 '만62세남자',
 '만63세남자',
 '만64세남자',
 '만65세남자',
 '만66세남자',
 '만67세남자',
 '만68세남자',
 '만69세남자',
 '만70세남자',
 '만71세남자',
 '만72세남자',
 '만73세남자',
 '만74세남자',
 '만75세남자',
 '만76세남자',
 '만77세남자',
 '만78세남자',
 '만79세남자',
 '만80세남자',
 '만81세남자',
 '만82세남자',
 '만83세남자',
 '만84세남자',
 '만85세남자',
 '만86세남자',
 '만87세남

In [67]:
# 선택 컬럼 이름
cols = ['시도명','시군구명','계','남자','여자','10대 이하','10대','20대','30대','40대','50대','60대','70대','80대','90대','100대 이상']

# 선택된 컬럼만 남기고 나머지 컬럼 삭제
df1 = df.select(cols)
df1.show()

+----------+--------+----+----+----+---------+----+----+----+----+----+----+----+----+----+----------+
|    시도명|시군구명|  계|남자|여자|10대 이하|10대|20대|30대|40대|50대|60대|70대|80대|90대|100대 이상|
+----------+--------+----+----+----+---------+----+----+----+----+----+----+----+----+----+----------+
|서울특별시|  종로구|1968| 759|1209|        0|   2| 252| 428| 345| 278| 264| 206| 158|  33|         2|
|서울특별시|  종로구|2293| 913|1380|        0|  12| 321| 623| 394| 302| 272| 184| 148|  34|         3|
|서울특별시|  종로구| 553| 264| 289|        1|   1|  64|  93|  92|  73|  99|  68|  49|  13|         0|
|서울특별시|  종로구|1667| 709| 958|        0|  12| 224| 350| 262| 260| 281| 146| 112|  19|         1|
|서울특별시|  종로구|2328| 934|1394|        0|  20| 229| 415| 331| 382| 407| 274| 214|  50|         6|
|서울특별시|  종로구| 704| 272| 432|        0|   1|  21|  94|  67| 111| 170| 126|  95|  18|         1|
|서울특별시|  종로구|1632| 648| 984|        1|   3| 217| 353| 244| 262| 243| 180| 114|  15|         0|
|서울특별시|  종로구| 929| 386| 543|        0|   2|  89| 194|

In [70]:
# 선택 컬럼 이름
cols = ['시도명','계','남자','여자','10대 이하','10대','20대','30대','40대','50대','60대','70대','80대','90대','100대 이상']

# 선택된 컬럼만 남기고 나머지 컬럼 삭제
df2 = df.select(cols)
df2.show()

+----------+----+----+----+---------+----+----+----+----+----+----+----+----+----+----------+
|    시도명|  계|남자|여자|10대 이하|10대|20대|30대|40대|50대|60대|70대|80대|90대|100대 이상|
+----------+----+----+----+---------+----+----+----+----+----+----+----+----+----+----------+
|서울특별시|1968| 759|1209|        0|   2| 252| 428| 345| 278| 264| 206| 158|  33|         2|
|서울특별시|2293| 913|1380|        0|  12| 321| 623| 394| 302| 272| 184| 148|  34|         3|
|서울특별시| 553| 264| 289|        1|   1|  64|  93|  92|  73|  99|  68|  49|  13|         0|
|서울특별시|1667| 709| 958|        0|  12| 224| 350| 262| 260| 281| 146| 112|  19|         1|
|서울특별시|2328| 934|1394|        0|  20| 229| 415| 331| 382| 407| 274| 214|  50|         6|
|서울특별시| 704| 272| 432|        0|   1|  21|  94|  67| 111| 170| 126|  95|  18|         1|
|서울특별시|1632| 648| 984|        1|   3| 217| 353| 244| 262| 243| 180| 114|  15|         0|
|서울특별시| 929| 386| 543|        0|   2|  89| 194| 157| 170| 140| 107|  59|  11|         0|
|서울특별시|3962|2736|1226|       

In [69]:
df1.groupBy("시도명").count().collect()

[Row(시도명='전라북도', count=243),
 Row(시도명='광주광역시', count=97),
 Row(시도명='대전광역시', count=82),
 Row(시도명='울산광역시', count=56),
 Row(시도명='인천광역시', count=159),
 Row(시도명='충청북도', count=153),
 Row(시도명='충청남도', count=210),
 Row(시도명='부산광역시', count=205),
 Row(시도명='세종특별자치시', count=24),
 Row(시도명='전라남도', count=323),
 Row(시도명='경기도', count=572),
 Row(시도명='강원특별자치도', count=194),
 Row(시도명='서울특별시', count=426),
 Row(시도명='경상남도', count=310),
 Row(시도명='대구광역시', count=152),
 Row(시도명='제주특별자치도', count=43),
 Row(시도명='경상북도', count=335)]

In [71]:
df2.describe().show()

+-------+--------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|        시도명|               계|              남자|              여자|         10대 이하|              10대|              20대|              30대|              40대|              50대|              60대|              70대|              80대|              90대|        100대 이상|
+-------+--------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|  count|          3584|             3584|              3584|              3584|              3584|              3584|              3584|              35

In [72]:
df2.groupBy("시도명").sum("10대 이하","10대","20대","30대","40대","50대","60대","70대","80대","90대","100대 이상").show()

+--------------+--------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------------+
|        시도명|sum(10대 이하)|sum(10대)|sum(20대)|sum(30대)|sum(40대)|sum(50대)|sum(60대)|sum(70대)|sum(80대)|sum(90대)|sum(100대 이상)|
+--------------+--------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------------+
|      전라북도|            82|     1356|    42330|    42809|    44421|    69192|    80418|    52962|    44255|     7949|            231|
|    광주광역시|            69|     1028|    45787|    46154|    37670|    44653|    46598|    27816|    17741|     2515|             80|
|    대전광역시|            85|     2089|    66107|    52434|    35402|    43073|    47063|    26123|    16237|     2508|             71|
|    울산광역시|            44|      303|    22568|    28050|    26494|    33579|    36770|    19139|     9614|     1160|             19|
|    인천광역시|           179|     1828|    71249|    90827|    73896|    9

In [73]:
df2.groupBy("시도명").sum("계","남자","여자").show()

+--------------+-------+---------+---------+
|        시도명|sum(계)|sum(남자)|sum(여자)|
+--------------+-------+---------+---------+
|      전라북도| 386005|   192959|   193046|
|    광주광역시| 270111|   136441|   133670|
|    대전광역시| 291192|   150239|   140953|
|    울산광역시| 177740|   100620|    77120|
|    인천광역시| 515310|   274162|   241148|
|      충청북도| 347596|   186954|   160642|
|      충청남도| 456625|   251223|   205402|
|    부산광역시| 647458|   305501|   341957|
|세종특별자치시|  55814|    29583|    26231|
|      전라남도| 428667|   217093|   211574|
|        경기도|2229876|  1229559|  1000317|
|강원특별자치도| 342441|   177202|   165239|
|    서울특별시|1984710|   954889|  1029821|
|      경상남도| 619398|   318581|   300817|
|    대구광역시| 429091|   205449|   223642|
|제주특별자치도| 133911|    69722|    64189|
|      경상북도| 581929|   298327|   283602|
+--------------+-------+---------+---------+



In [86]:
df4 = df2.select("시도명","계","남자","여자")

In [87]:
df4.show()

+----------+----+----+----+
|    시도명|  계|남자|여자|
+----------+----+----+----+
|서울특별시|1968| 759|1209|
|서울특별시|2293| 913|1380|
|서울특별시| 553| 264| 289|
|서울특별시|1667| 709| 958|
|서울특별시|2328| 934|1394|
|서울특별시| 704| 272| 432|
|서울특별시|1632| 648| 984|
|서울특별시| 929| 386| 543|
|서울특별시|3962|2736|1226|
|서울특별시|2613|1499|1114|
|서울특별시|2790|1322|1468|
|서울특별시|5724|2850|2874|
|서울특별시|1598| 949| 649|
|서울특별시|2029|1133| 896|
|서울특별시| 940| 465| 475|
|서울특별시|1495| 739| 756|
|서울특별시|4726|2336|2390|
|서울특별시| 755| 352| 403|
|서울특별시|1546| 974| 572|
|서울특별시| 995| 550| 445|
+----------+----+----+----+
only showing top 20 rows



In [92]:
# 컬럼명 변경
newcol = ["region","total","man","woman"]
df5 = df4.toDF(*newcol)
df5.show()

+----------+-----+----+-----+
|    region|total| man|woman|
+----------+-----+----+-----+
|서울특별시| 1968| 759| 1209|
|서울특별시| 2293| 913| 1380|
|서울특별시|  553| 264|  289|
|서울특별시| 1667| 709|  958|
|서울특별시| 2328| 934| 1394|
|서울특별시|  704| 272|  432|
|서울특별시| 1632| 648|  984|
|서울특별시|  929| 386|  543|
|서울특별시| 3962|2736| 1226|
|서울특별시| 2613|1499| 1114|
|서울특별시| 2790|1322| 1468|
|서울특별시| 5724|2850| 2874|
|서울특별시| 1598| 949|  649|
|서울특별시| 2029|1133|  896|
|서울특별시|  940| 465|  475|
|서울특별시| 1495| 739|  756|
|서울특별시| 4726|2336| 2390|
|서울특별시|  755| 352|  403|
|서울특별시| 1546| 974|  572|
|서울특별시|  995| 550|  445|
+----------+-----+----+-----+
only showing top 20 rows



In [93]:
group_df = df5.groupBy("region").agg(sum(col("total")).alias("total"),sum(col("man")).alias("man_total"),sum(col("woman")).alias("woman_total"))
group_df.show()

+--------------+-------+---------+-----------+
|        region|  total|man_total|woman_total|
+--------------+-------+---------+-----------+
|      전라북도| 386005|   192959|     193046|
|    광주광역시| 270111|   136441|     133670|
|    대전광역시| 291192|   150239|     140953|
|    울산광역시| 177740|   100620|      77120|
|    인천광역시| 515310|   274162|     241148|
|      충청북도| 347596|   186954|     160642|
|      충청남도| 456625|   251223|     205402|
|    부산광역시| 647458|   305501|     341957|
|세종특별자치시|  55814|    29583|      26231|
|      전라남도| 428667|   217093|     211574|
|        경기도|2229876|  1229559|    1000317|
|강원특별자치도| 342441|   177202|     165239|
|    서울특별시|1984710|   954889|    1029821|
|      경상남도| 619398|   318581|     300817|
|    대구광역시| 429091|   205449|     223642|
|제주특별자치도| 133911|    69722|      64189|
|      경상북도| 581929|   298327|     283602|
+--------------+-------+---------+-----------+



In [94]:
# 내림차순 정렬
group_df = group_df.sort(group_df.total.desc())
group_df.show()

+--------------+-------+---------+-----------+
|        region|  total|man_total|woman_total|
+--------------+-------+---------+-----------+
|        경기도|2229876|  1229559|    1000317|
|    서울특별시|1984710|   954889|    1029821|
|    부산광역시| 647458|   305501|     341957|
|      경상남도| 619398|   318581|     300817|
|      경상북도| 581929|   298327|     283602|
|    인천광역시| 515310|   274162|     241148|
|      충청남도| 456625|   251223|     205402|
|    대구광역시| 429091|   205449|     223642|
|      전라남도| 428667|   217093|     211574|
|      전라북도| 386005|   192959|     193046|
|      충청북도| 347596|   186954|     160642|
|강원특별자치도| 342441|   177202|     165239|
|    대전광역시| 291192|   150239|     140953|
|    광주광역시| 270111|   136441|     133670|
|    울산광역시| 177740|   100620|      77120|
|제주특별자치도| 133911|    69722|      64189|
|세종특별자치시|  55814|    29583|      26231|
+--------------+-------+---------+-----------+



In [104]:
cols = ["total","man_total","woman_total"]
for i in cols:
    group_df = group_df.withColumn(i, group_df[i].cast('int'))

In [107]:
group_df.printSchema()

root
 |-- region: string (nullable = true)
 |-- total: integer (nullable = true)
 |-- man_total: integer (nullable = true)
 |-- woman_total: integer (nullable = true)



In [112]:
group_df.describe().show()

+-------+--------------+-----------------+------------------+-----------------+
|summary|        region|            total|         man_total|      woman_total|
+-------+--------------+-----------------+------------------+-----------------+
|  count|            17|               17|                17|               17|
|   mean|          null|582227.8823529412|          299912.0|282315.8823529412|
| stddev|          null|598409.6210366569|312909.74144351913|288105.0102744402|
|    min|강원특별자치도|            55814|             29583|            26231|
|    max|      충청북도|          2229876|           1229559|          1029821|
+-------+--------------+-----------------+------------------+-----------------+



In [111]:
from pyspark.sql.functions import max
group_df.show()
group_df.select(max(group_df.woman_total).alias("여성 최대 1인가구수")).show()
group_df.filter(group_df.woman_total == 1029821).show()

+--------------+-------+---------+-----------+
|        region|  total|man_total|woman_total|
+--------------+-------+---------+-----------+
|        경기도|2229876|  1229559|    1000317|
|    서울특별시|1984710|   954889|    1029821|
|    부산광역시| 647458|   305501|     341957|
|      경상남도| 619398|   318581|     300817|
|      경상북도| 581929|   298327|     283602|
|    인천광역시| 515310|   274162|     241148|
|      충청남도| 456625|   251223|     205402|
|    대구광역시| 429091|   205449|     223642|
|      전라남도| 428667|   217093|     211574|
|      전라북도| 386005|   192959|     193046|
|      충청북도| 347596|   186954|     160642|
|강원특별자치도| 342441|   177202|     165239|
|    대전광역시| 291192|   150239|     140953|
|    광주광역시| 270111|   136441|     133670|
|    울산광역시| 177740|   100620|      77120|
|제주특별자치도| 133911|    69722|      64189|
|세종특별자치시|  55814|    29583|      26231|
+--------------+-------+---------+-----------+

+-------------------+
|여성 최대 1인가구수|
+-------------------+
|            1029821|
+--------------

In [114]:
from pyspark.sql.functions import avg
group_df.select(avg(group_df.total).alias("총 1인가구수 평균")).show()

+-----------------+
|총 1인가구수 평균|
+-----------------+
|582227.8823529412|
+-----------------+



In [119]:
from pyspark.sql.functions import when
group_df.select(group_df.region.alias("시도명"), group_df.total.alias("총 1인가구수"),
                when(group_df.total > 582228, "평균초과").
                when(group_df.total < 582228, "평균미만").
                otherwise("평균").alias("결과")).show(17)

+--------------+------------+--------+
|        시도명|총 1인가구수|    결과|
+--------------+------------+--------+
|        경기도|     2229876|평균초과|
|    서울특별시|     1984710|평균초과|
|    부산광역시|      647458|평균초과|
|      경상남도|      619398|평균초과|
|      경상북도|      581929|평균미만|
|    인천광역시|      515310|평균미만|
|      충청남도|      456625|평균미만|
|    대구광역시|      429091|평균미만|
|      전라남도|      428667|평균미만|
|      전라북도|      386005|평균미만|
|      충청북도|      347596|평균미만|
|강원특별자치도|      342441|평균미만|
|    대전광역시|      291192|평균미만|
|    광주광역시|      270111|평균미만|
|    울산광역시|      177740|평균미만|
|제주특별자치도|      133911|평균미만|
|세종특별자치시|       55814|평균미만|
+--------------+------------+--------+

